In [0]:
from google.colab import drive, files
drive.mount('/content/drive')

drive_dir = '/content/drive/My Drive/'
root_dir = '/content'

%cd "{root_dir}"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content


In [0]:
repo_dir = "/content/repo"
!git clone https://github.com/xelav/Insider-Threat-CERT "$repo_dir"
! 

import sys
sys.path.append(repo_dir)

Cloning into '/content/repo'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 213 (delta 142), reused 147 (delta 86), pack-reused 0
Receiving objects: 100% (213/213), 1.38 MiB | 2.69 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [0]:
%cd "{repo_dir}"

!git checkout new-cnn-parameters


# !git fetch
# !git reset origin/master --hard
# !git pull
# !git checkout new-cnn-parameters
%cd "{root_dir}"

/content/repo
Branch 'new-cnn-parameters' set up to track remote branch 'new-cnn-parameters' from 'origin'.
Switched to a new branch 'new-cnn-parameters'
/content


In [0]:
!pip install pytorch-ignite
%pip install wandb -q

import wandb
wandb.login()

     |████████████████████████████████| 112kB 4.9MB/s 
     |████████████████████████████████| 1.4MB 4.9MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 460kB 54.5MB/s 
     |████████████████████████████████| 102kB 13.6MB/s 
     |████████████████████████████████| 112kB 54.4MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 


<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [0]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from pathlib import Path
import pandas as pd
import pickle
import numpy as np
import shutil
# from tqdm import tqdm_notebook as tqdm

import torch.utils.data
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator, Engine
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import EarlyStopping
from ignite.metrics import Accuracy, Loss

from src.models import InsiderClassifier, LSTM_Encoder
from src.params import get_params
from src.dataset import CertDataset, create_data_loaders
from src.cnn_trainer import *

device = 'cuda'
params = get_params()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
output_dir = Path(f'{drive_dir}/Datasets/CERT_output')
answers_dir = Path(f"{drive_dir}/Datasets/CERT/answers")
main_answers_file = answers_dir / "insiders.csv"

# lstm_checkpoint = output_dir / 'checkpoints/lstm/final2-nll/final_model_3040.pth'
lstm_checkpoint = output_dir / 'checkpoints/lstm/base-veryfinal2/best_model_498_non_pad_accuracy=0.7740616312925708.pth'
assert(lstm_checkpoint.is_file())

In [0]:
actions, targets = CertDataset.prepare_dataset(
    output_dir / 'aggregated.pkl',
    main_answers_file,
    min_length=params['min_seq_length'],
    max_length=params['seq_length'])

cert_dataset = CertDataset(actions, targets)
train_loader, val_loader = create_data_loaders(
    cert_dataset,
    validation_split=0.3,
    random_seed=0,
    batch_size=1024)

In [0]:
def nest_dict(flat_dict, sep='.'):
    """
    credit to https://stackoverflow.com/a/35508197
    Return nested dict by splitting the keys on a delimiter.

    >>> from pprint import pprint
    >>> pprint(nest_dict({'title': 'foo', 'author_name': 'stretch',
    ... 'author_zipcode': '06901'}))
    {'author': {'name': 'stretch', 'zipcode': '06901'}, 'title': 'foo'}
    """
    tree = {}
    for key, val in flat_dict.items():
        t = tree
        prev = None
        for part in key.split(sep):
            if prev is not None:
                t = t.setdefault(prev, {})
            prev = part
        else:
            t.setdefault(prev, val)
    return tree

import collections.abc
def deep_update(d, u):
    # credit to https://stackoverflow.com/a/3233356
    for k, v in u.items():
        if isinstance(v, dict):
            d[k] = deep_update(d.get(k, {}), v)
        else:
            d[k] = v
    return d

def calculate_weights(targets, weight_beta=0.1):

    anomalous_num = targets.sum()
    overall_num = targets.shape[0]
    non_anomalous_num = overall_num - anomalous_num

    anomalous_weight = 1 + (non_anomalous_num - anomalous_num) / (weight_beta * non_anomalous_num)

    weight = torch.tensor([1, anomalous_weight]).float()

    return weight

def train_wrapper():

    default_params = get_params()
    default_params['train']['cnn_classifier']['num_epochs'] = 1000
    default_params['model']['lstm_encoder']['embedding_size'] = None

    # Initialize a new wandb run
    wandb.init(config=default_params)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    params = wandb.config
    params = nest_dict(dict(params))
    params['train']['cnn_classifier'] = {**default_params['train']['cnn_classifier'], **params['train']['cnn_classifier']}
    params['model']['cnn_classifier'] = {**default_params['model']['cnn_classifier'], **params['model']['cnn_classifier']}
    params['train'] = {**default_params['train'], **params['train']}
    params['model'] = {**default_params['model'], **params['model']}
    print(params)

    train_params = params['train']['cnn_classifier']

    model = InsiderClassifier(params['model'], lstm_checkpoint)

    # 
    # wandb.watch(model)
    # print("Run dir:  ", wandb.run.dir)

    class_weights = calculate_weights(targets, train_params['weighting_beta']).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=train_params['lr'])

    train_engine = create_supervised_trainer(
                                            model, optimizer, criterion, device=device,
                                            prepare_batch=prepare_batch,
                                        )

    if train_params['lr_annealing']:
        scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-1, 1e-3, len(train_loader))
        train_engine.add_event_handler(Events.ITERATION_STARTED, scheduler)

    val_engine = create_supervised_evaluator(
            model, device=device,
            prepare_batch=prepare_batch,
            metrics={},
            criterion=criterion,
    )

    handler = EarlyStopping(
        patience=20,
        score_function=lambda _: val_engine.state.metrics['roc_auc'],
        trainer=train_engine)
    val_engine.add_event_handler(Events.COMPLETED, handler)

    @train_engine.on(Events.STARTED)
    def initial_validation(trainer):
        print('Initial validation run:')
        val_engine.train_epoch = 0
        val_engine.run(val_loader)

        wandb.run.summary["best_val_roc_auc"] = val_engine.state.metrics['roc_auc']
        wandb.run.summary["best_val_loss"] = val_engine.state.metrics['loss']


    @train_engine.on(Events.EPOCH_COMPLETED)
    def log_training_results(trainer):
        val_engine.train_epoch = train_engine.state.epoch
        val_engine.run(val_loader)

        if val_engine.state.metrics['roc_auc'] > wandb.run.summary["best_val_roc_auc"]:
            wandb.run.summary["best_val_roc_auc"] = val_engine.state.metrics['roc_auc']
            wandb.run.summary["best_val_loss"] = val_engine.state.metrics['loss']
            wandb.run.summary["best_train_roc_auc"] = train_engine.state.metrics['roc_auc']
            wandb.run.summary["best_train_loss"] = train_engine.state.metrics['loss']

            wandb.save(str(Path(wandb.run.dir) / 'checkpoints/best*'))

    torch.manual_seed(train_params['manual_seed'])
    train_engine.run(train_loader, max_epochs=train_params['num_epochs'])

In [0]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'best_val_roc_auc',
      'goal': 'maximize'   
    },
    'parameters': {
                'model.cnn_classifier.activation': {
                    'values': ['relu', 'tanh', 'elu', 'leaky_relu']
                },
                'model.cnn_classifier.apply_batch_norm': {
                    'values': [True, False]
                },
                'model.cnn_classifier.dropout': {
                    'values': [0, 0.5]
                },

                
                'train.cnn_classifier.weighting_beta': {
                    'values': [1e6, 1, 0.5, 0.1, 0.05]
                },
                'train.cnn_classifier.weight_decay': {
                    'values': [0, 0.01, 0.001]
                },
                'train.cnn_classifier.lr_annealing': {
                    'values': [True, False]
                },
                'train.cnn_classifier.lr': {
                    'values': [1e-2, 1e-3, 1e-4]
                },
                'train.cnn_classifier.manual_seed': {
                    'values': [0, 1, 2, 3, 4, 5]
                }
        
    }
}

# sweep_id = wandb.sweep(sweep_config, entity='xelav', project="cert-cnn")

# Initialize a new sweep
# Arguments:
#     – sweep_id: the sweep_id to run - this was returned above by wandb.sweep()
#     – function: function that defines your model architecture and trains it
sweep_id = 'j6zxr02n'
wandb.agent(sweep_id, train_wrapper)

Train Results - Avg loss: 7.336583, ROC AUC: 0.620925
Validation Results - Avg loss: 1.924026, ROC AUC: 0.682095
Train Results - Avg loss: 11.337890, ROC AUC: 0.588628
Validation Results - Avg loss: 1.694346, ROC AUC: 0.686366
wandb: Agent Finished Run: aqu61fs2 

wandb: Agent Starting Run: 71msvdgv with config:
	model.cnn_classifier.activation: relu
	model.cnn_classifier.apply_batch_norm: False
	model.cnn_classifier.dropout: 0.5
	train.cnn_classifier.lr: 0.0001
	train.cnn_classifier.lr_annealing: False
	train.cnn_classifier.manual_seed: 0
	train.cnn_classifier.weight_decay: 0
	train.cnn_classifier.weighting_beta: 0.1
wandb: Agent Started Run: 71msvdgv


{'model': {'cnn_classifier': {'conv1_filters': 32, 'conv1_kernel_size': 5, 'conv2_filters': 64, 'conv2_kernel_size': 5, 'max_seq_length': 200, 'lstm_hidden_size': 40, 'activation': 'relu', 'apply_batch_norm': False, 'dropout': 0.5}, 'lstm_encoder': {'input_size': 64, 'hidden_size': 40, 'embedding_size': None, 'num_layers': 3, 'dropout': 0.5}}, 'train': {'lstm_encoder': {'num_epochs': 200, 'learning_rate': 0.001, 'batch_size': 1024, 'manual_seed': 0}, 'cnn_classifier': {'num_epochs': 1000, 'learning_rate': 0.001, 'batch_size': 1024, 'manual_seed': 0, 'weighting_beta': 0.1, 'lr': 0.0001, 'lr_annealing': False, 'weight_decay': 0}}, 'min_seq_length': 50, 'seq_length': 200}
Initial validation run:
Validation Results - Avg loss: 0.592434, ROC AUC: 0.740571
Train Results - Avg loss: 0.181222, ROC AUC: 0.550462
Validation Results - Avg loss: 0.395045, ROC AUC: 0.792687
Train Results - Avg loss: 0.178074, ROC AUC: 0.556727
Validation Results - Avg loss: 0.355897, ROC AUC: 0.820731
Train Results

/usr/local/lib/python3.6/dist-packages/ignite/metrics/epoch_metric.py:79: RuntimeWarning: Probably, there can be a problem with `compute_fn`:
 Only one class present in y_true. ROC AUC score is not defined in that case..
  RuntimeWarning)


Train Results - Avg loss: 0.100904, ROC AUC: 0.928966
Validation Results - Avg loss: 0.159661, ROC AUC: 0.907693
Train Results - Avg loss: 0.099205, ROC AUC: 0.932949
Validation Results - Avg loss: 0.186255, ROC AUC: 0.907210
Train Results - Avg loss: 0.099900, ROC AUC: 0.933383
Validation Results - Avg loss: 0.159907, ROC AUC: 0.907472
Train Results - Avg loss: 0.098862, ROC AUC: 0.932964
Validation Results - Avg loss: 0.172469, ROC AUC: 0.910464
Train Results - Avg loss: 0.097215, ROC AUC: 0.934391
Validation Results - Avg loss: 0.166256, ROC AUC: 0.912358
Train Results - Avg loss: 0.097087, ROC AUC: 0.934339
Validation Results - Avg loss: 0.172662, ROC AUC: 0.910308
Train Results - Avg loss: 0.099079, ROC AUC: 0.934152
Validation Results - Avg loss: 0.161321, ROC AUC: 0.911637
Train Results - Avg loss: 0.097223, ROC AUC: 0.935238
Validation Results - Avg loss: 0.156265, ROC AUC: 0.909333
Train Results - Avg loss: 0.095761, ROC AUC: 0.933380
Validation Results - Avg loss: 0.155397, R

/usr/local/lib/python3.6/dist-packages/ignite/metrics/epoch_metric.py:79: RuntimeWarning: Probably, there can be a problem with `compute_fn`:
 Only one class present in y_true. ROC AUC score is not defined in that case..
  RuntimeWarning)


...

/usr/local/lib/python3.6/dist-packages/ignite/metrics/epoch_metric.py:79: RuntimeWarning: Probably, there can be a problem with `compute_fn`:
 Only one class present in y_true. ROC AUC score is not defined in that case..
  RuntimeWarning)


Train Results - Avg loss: 0.030120, ROC AUC: 0.960516
Validation Results - Avg loss: 0.032819, ROC AUC: 0.939858
Train Results - Avg loss: 0.029769, ROC AUC: 0.962110
Validation Results - Avg loss: 0.034117, ROC AUC: 0.940224
Train Results - Avg loss: 0.030224, ROC AUC: 0.961488
Validation Results - Avg loss: 0.034799, ROC AUC: 0.929074
Train Results - Avg loss: 0.029260, ROC AUC: 0.964943
Validation Results - Avg loss: 0.032608, ROC AUC: 0.940498
Train Results - Avg loss: 0.029050, ROC AUC: 0.963415
Validation Results - Avg loss: 0.033106, ROC AUC: 0.939819
Train Results - Avg loss: 0.028435, ROC AUC: 0.962992
Validation Results - Avg loss: 0.032196, ROC AUC: 0.938816
Train Results - Avg loss: 0.028200, ROC AUC: 0.968004
Validation Results - Avg loss: 0.032094, ROC AUC: 0.941537
Train Results - Avg loss: 0.028880, ROC AUC: 0.961106
Validation Results - Avg loss: 0.031778, ROC AUC: 0.938467
Train Results - Avg loss: 0.027564, ROC AUC: 0.969176
Validation Results - Avg loss: 0.031863, R

{'model': {'cnn_classifier': {'conv1_filters': 32, 'conv1_kernel_size': 5, 'conv2_filters': 64, 'conv2_kernel_size': 5, 'max_seq_length': 200, 'lstm_hidden_size': 40, 'activation': 'tanh', 'apply_batch_norm': True, 'dropout': 0.5}, 'lstm_encoder': {'input_size': 64, 'hidden_size': 40, 'embedding_size': None, 'num_layers': 3, 'dropout': 0.5}}, 'train': {'lstm_encoder': {'num_epochs': 200, 'learning_rate': 0.001, 'batch_size': 1024, 'manual_seed': 0}, 'cnn_classifier': {'num_epochs': 1000, 'learning_rate': 0.001, 'batch_size': 1024, 'manual_seed': 0, 'weighting_beta': 0.5, 'lr': 0.0001, 'lr_annealing': False, 'weight_decay': 0.01}}, 'min_seq_length': 50, 'seq_length': 200}
Initial validation run:
Validation Results - Avg loss: 0.629761, ROC AUC: 0.703068
Train Results - Avg loss: 0.100321, ROC AUC: 0.577559
Validation Results - Avg loss: 0.342377, ROC AUC: 0.885477
Train Results - Avg loss: 0.051175, ROC AUC: 0.890632
Validation Results - Avg loss: 0.576492, ROC AUC: 0.893220
Train Resul

/usr/local/lib/python3.6/dist-packages/ignite/metrics/epoch_metric.py:79: RuntimeWarning: Probably, there can be a problem with `compute_fn`:
 Only one class present in y_true. ROC AUC score is not defined in that case..
  RuntimeWarning)


Train Results - Avg loss: 0.036049, ROC AUC: 0.938878
Validation Results - Avg loss: 2.115595, ROC AUC: 0.903723
Train Results - Avg loss: 0.036655, ROC AUC: 0.935721
Validation Results - Avg loss: 2.557754, ROC AUC: 0.915334
Train Results - Avg loss: 0.036388, ROC AUC: 0.941523
Validation Results - Avg loss: 2.324713, ROC AUC: 0.916516
Train Results - Avg loss: 0.035072, ROC AUC: 0.947446
Validation Results - Avg loss: 2.253552, ROC AUC: 0.895537
Train Results - Avg loss: 0.033887, ROC AUC: 0.947860
Validation Results - Avg loss: 2.564891, ROC AUC: 0.898651
Train Results - Avg loss: 0.034062, ROC AUC: 0.950622
Validation Results - Avg loss: 2.363225, ROC AUC: 0.907268
Train Results - Avg loss: 0.034868, ROC AUC: 0.949261
Validation Results - Avg loss: 2.338752, ROC AUC: 0.907581
Train Results - Avg loss: 0.036088, ROC AUC: 0.944358
Validation Results - Avg loss: 2.395515, ROC AUC: 0.917863
Train Results - Avg loss: 0.034735, ROC AUC: 0.948902
Validation Results - Avg loss: 2.443713, R